In [1]:
import json
from typing import Any, Dict

from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor
from opentelemetry.trace import Status, StatusCode, set_tracer_provider

from openinference.instrumentation import TraceConfig
from openinference.instrumentation.config import (
    OITracer,
    chain,
    get_input_value_and_mime_type,
    get_openinference_span_kind,
    get_output_value_and_mime_type,
)
from openinference.semconv.resource import ResourceAttributes

In [2]:
endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={ResourceAttributes.PROJECT_NAME: "openinference-tracer"})
tracer_provider = TracerProvider(resource=resource)
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
tracer_provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))
set_tracer_provider(tracer_provider)
tracer = OITracer(wrapped=tracer_provider.get_tracer(__name__), config=TraceConfig())

## Chains

### Context Manager

In [3]:
with tracer.start_as_current_span(
    "chain-span-with-plain-text-io",
    openinference_span_kind="chain",
) as span:
    span.set_input("input")
    span.set_output("output")
    span.set_status(Status(StatusCode.OK))

{
    "name": "chain-span-with-plain-text-io",
    "context": {
        "trace_id": "0xeee32ec86ffa59d8baa3ab0394680917",
        "span_id": "0xa85238d48d401a03",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.377311Z",
    "end_time": "2025-01-16T06:15:45.377354Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "input",
        "input.mime_type": "text/plain",
        "output.value": "output",
        "output.mime_type": "text/plain"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}


In [4]:
with tracer.start_as_current_span(
    "chain-span-with-json-io",
    openinference_span_kind="chain",
) as span:
    span.set_input(
        json.dumps({"input-key": "input-value"}),
        mime_type="application/json",
    )
    span.set_output(
        json.dumps({"output-key": "output-value"}),
        mime_type="application/json",
    )
    span.set_status(Status(StatusCode.OK))

{
    "name": "chain-span-with-json-io",
    "context": {
        "trace_id": "0x0ccbe14ecd943a88ce476c88adc04b81",
        "span_id": "0x59e61cad8e902931",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.409922Z",
    "end_time": "2025-01-16T06:15:45.409998Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "{\"input-key\": \"input-value\"}",
        "input.mime_type": "application/json",
        "output.value": "{\"output-key\": \"output-value\"}",
        "output.mime_type": "application/json"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}


In [5]:
with tracer.start_as_current_span(
    "chain-span-with-attribute-getters",
    attributes={
        **get_openinference_span_kind("chain"),
        **get_input_value_and_mime_type("input"),
    },
) as span:
    span.set_attributes(get_output_value_and_mime_type("output"))
    span.set_status(Status(StatusCode.OK))

{
    "name": "chain-span-with-attribute-getters",
    "context": {
        "trace_id": "0xcba92cde3dbe20b802afeb24db589aa6",
        "span_id": "0xf890625f1204477a",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.455835Z",
    "end_time": "2025-01-16T06:15:45.455961Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "input.value": "input",
        "input.mime_type": "text/plain",
        "output.value": "output",
        "output.mime_type": "text/plain",
        "openinference.span.kind": "CHAIN"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}


### Decorator

In [6]:
@chain
def decorated_chain_with_plain_text_output(input: str) -> str:
    return "output"


decorated_chain_with_plain_text_output("input")

{
    "name": "decorated_chain_with_plain_text_output",
    "context": {
        "trace_id": "0xe5223361341698eb77393d944f2c1e14",
        "span_id": "0x3bab97577210662f",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.482804Z",
    "end_time": "2025-01-16T06:15:45.482851Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "{\"input\": \"input\"}",
        "input.mime_type": "application/json",
        "output.value": "output",
        "output.mime_type": "text/plain"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}


'output'

In [7]:
@chain
def decorated_chain_with_json_output(input: str) -> Dict[str, Any]:
    return {"output": "output"}


decorated_chain_with_json_output("input")

{
    "name": "decorated_chain_with_json_output",
    "context": {
        "trace_id": "0x428fbcd543e74e4bfc49062ea78ddf7a",
        "span_id": "0x0e31b128f2990336",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.498932Z",
    "end_time": "2025-01-16T06:15:45.498981Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "{\"input\": \"input\"}",
        "input.mime_type": "application/json",
        "output.value": "{\"output\": \"output\"}",
        "output.mime_type": "application/json"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}


{'output': 'output'}

In [8]:
@chain()
def decorated_chain_with_no_parameters(input: str) -> Dict[str, Any]:
    return {"output": "output"}


decorated_chain_with_no_parameters("input")

{
    "name": "decorated_chain_with_no_parameters",
    "context": {
        "trace_id": "0x2ab2f3d0e4db8a873ea9438aa6f8e8e4",
        "span_id": "0x53d4eb47a692552d",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.510554Z",
    "end_time": "2025-01-16T06:15:45.510609Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "{\"input\": \"input\"}",
        "input.mime_type": "application/json",
        "output.value": "{\"output\": \"output\"}",
        "output.mime_type": "application/json"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}


{'output': 'output'}

In [9]:
@chain(name="decorated-chain-with-overriden-name")
def this_name_should_be_overriden(input: str) -> Dict[str, Any]:
    return {"output": "output"}


this_name_should_be_overriden("input")

{
    "name": "decorated-chain-with-overriden-name",
    "context": {
        "trace_id": "0xd9478cc36c4c260306b436d50af820b3",
        "span_id": "0x6f8d2d6b20d63313",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.519739Z",
    "end_time": "2025-01-16T06:15:45.519789Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "{\"input\": \"input\"}",
        "input.mime_type": "application/json",
        "output.value": "{\"output\": \"output\"}",
        "output.mime_type": "application/json"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}


{'output': 'output'}

In [10]:
@chain
async def decorated_async_chain(input: str) -> str:
    return "output"


await decorated_async_chain("input")  # type: ignore[top-level-await]

{
    "name": "decorated_async_chain",
    "context": {
        "trace_id": "0xb4516b4b9f75410e5b25d5986a677ef2",
        "span_id": "0x695a4df46529b5b3",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.528912Z",
    "end_time": "2025-01-16T06:15:45.528952Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "{\"input\": \"input\"}",
        "input.mime_type": "application/json",
        "output.value": "output",
        "output.mime_type": "text/plain"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}


'output'

In [11]:
@chain
def decorated_chain_with_error(input: str) -> str:
    raise ValueError("error")


try:
    decorated_chain_with_error("input")
except ValueError as e:
    print(e)

{
    "name": "decorated_chain_with_error",
    "context": {
        "trace_id": "0x6718666162aad2cd4487dd329b1986f4",
        "span_id": "0xd61d20a45a9e3bfe",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.540278Z",
    "end_time": "2025-01-16T06:15:45.541206Z",
    "status": {
        "status_code": "ERROR",
        "description": "ValueError: error"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "{\"input\": \"input\"}",
        "input.mime_type": "application/json"
    },
    "events": [
        {
            "name": "exception",
            "timestamp": "2025-01-16T06:15:45.541192Z",
            "attributes": {
                "exception.type": "ValueError",
                "exception.message": "error",
                "exception.stacktrace": "Traceback (most recent call last):\n  File \"/Users/xandersong/openinference/python/openinference-instrumentati

In [12]:
@chain
def decorated_chain_with_child_span(input: str) -> str:
    with tracer.start_as_current_span(
        "child-span",
        openinference_span_kind="chain",
        attributes=get_input_value_and_mime_type("child-span-input"),
    ) as child_span:
        output = "output"
        child_span.set_output(output)
        child_span.set_status(Status(StatusCode.OK))
        return output


decorated_chain_with_child_span("input")

{
    "name": "child-span",
    "context": {
        "trace_id": "0x1a0326daaae88a5f6bdfabb9adff0269",
        "span_id": "0xbeb154012cbd172f",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x634f4ca30bca35b6",
    "start_time": "2025-01-16T06:15:45.552864Z",
    "end_time": "2025-01-16T06:15:45.552889Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "child-span-input",
        "input.mime_type": "text/plain",
        "output.value": "output",
        "output.mime_type": "text/plain"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}
{
    "name": "decorated_chain_with_child_span",
    "context": {
        "trace_id": "0x1a0326daaae88a5f6bdfabb9adff0269",
        "span_id": "0x634f4ca30bca35b6",
        "trace_state"

'output'

In [13]:
@chain
def decorated_chain_with_child_span_error(input: str) -> str:
    with tracer.start_as_current_span(
        "child-span",
        openinference_span_kind="chain",
        attributes=get_input_value_and_mime_type("child-span-input"),
    ):
        raise ValueError("error")


try:
    decorated_chain_with_child_span_error("input")
except ValueError as e:
    print(e)

{
    "name": "child-span",
    "context": {
        "trace_id": "0xef25a9aeecfec7ba9371b09ffe24f8d6",
        "span_id": "0xf4bc769f20794b97",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x83e29d3fed0273c0",
    "start_time": "2025-01-16T06:15:45.569152Z",
    "end_time": "2025-01-16T06:15:45.569482Z",
    "status": {
        "status_code": "ERROR",
        "description": "ValueError: error"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "child-span-input",
        "input.mime_type": "text/plain"
    },
    "events": [
        {
            "name": "exception",
            "timestamp": "2025-01-16T06:15:45.569471Z",
            "attributes": {
                "exception.type": "ValueError",
                "exception.message": "error",
                "exception.stacktrace": "Traceback (most recent call last):\n  File \"/Users/xandersong/openinference/python/openinference-instrumentation/.venv/lib

In [14]:
class ChainRunner:
    @chain
    def decorated_chain_method(self, input: str) -> str:
        return "output"


chain_runner = ChainRunner()
chain_runner.decorated_chain_method("input")

{
    "name": "decorated_chain_method",
    "context": {
        "trace_id": "0xe81fde60ee5fb1e0f0bc7f7bb872d47c",
        "span_id": "0xe4ebb5740668983e",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-01-16T06:15:45.586472Z",
    "end_time": "2025-01-16T06:15:45.586509Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "openinference.span.kind": "CHAIN",
        "input.value": "{\"input\": \"input\"}",
        "input.mime_type": "application/json",
        "output.value": "output",
        "output.mime_type": "text/plain"
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "openinference.project.name": "openinference-tracer"
        },
        "schema_url": ""
    }
}


'output'